<a href="https://colab.research.google.com/github/vincent-grari/DIVERS/blob/main/Clara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import sys
print(sys.argv)
import os
import urllib
import os.path
import sklearn.preprocessing as preprocessing
from collections import namedtuple
import pandas as pd

['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-f8b6ec8e-2d31-4f44-9fc4-aa64861aa36f.json']


In [ ]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.2)


In [ ]:
!wget "http://grarivincent.com/research/baseINSEE.csv"

--2021-06-28 14:16:36--  http://grarivincent.com/research/baseINSEE.csv
Resolving grarivincent.com (grarivincent.com)... 217.160.0.183, 2001:8d8:100f:f000::2ce
Connecting to grarivincent.com (grarivincent.com)|217.160.0.183|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5107089 (4.9M) [text/csv]
Saving to: ‘baseINSEE.csv.2’

baseINSEE.csv.2     100%[===================>]   4.87M  5.49MB/s    in 0.9s    

2021-06-28 14:16:38 (5.49 MB/s) - ‘baseINSEE.csv.2’ saved [5107089/5107089]



In [ ]:
baseINSEE = (pd.read_csv("baseINSEE.csv", #names=column_names,
                    sep=r'\s*;\s*', engine='python', na_values=['NA']))

In [ ]:
baseINSEE['pol_insee_code']= baseINSEE['code_commune_INSEE']

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
def load_ICU_Pricing(path):
    #column_names = ["CalYear","Gender","Type","Category","Occupation","Age","Group1","Bonus","Poldur","Value","Adind","SubGroup2","Group2","Density","Y"]
    #input_data = (pd.read_csv(path, #names=column_names,
    #                           sep=r',', engine='python', keep_default_na=False, na_values=['NA']))
    
    path = "http://grarivincent.com/research/db4ModelFull2.csv"
    input_data = (pd.read_csv(path, #names=column_names,
                                sep=r',', engine='python', na_values=['NA']))
    input_data=input_data.drop(columns=['Unnamed: 0'])
    
    base=baseINSEE
    base = base[~base.pol_insee_code.str[:2].isin(["2A","2B"])]
    base["pol_insee_code"] = base["pol_insee_code"].astype(int)
    base_nd =  base.drop_duplicates(subset=['pol_insee_code'])

    input_data = input_data.drop_duplicates()
    input_data = input_data[~input_data.pol_insee_code.str[:2].isin(["2A","2B"])]
    input_data["pol_insee_code"] = input_data["pol_insee_code"].astype(int)

    input_data = input_data.merge(base_nd, how='inner', on="pol_insee_code")
    #input_data = input_data[input_data['claim_amount']<=5000]
    #input_data=input_data[input_data['claim_amount']!=0]
   
    #
    sensitive_attribs = ['Age']
    print(input_data.shape)
    G0 = (input_data.iloc[:, 37:180]) #137])
    #Z0 = pd.concat(Z0,input_data['long'])
    G0=G0.select_dtypes([np.number])
    G0 = G0.fillna(G0.mean())
    G=G0
    #Z = (Z0
    #    .drop(columns=['"REG"'
    #                   ])
    #    .fillna('Unknown').pipe(pd.get_dummies))

    y = input_data['claim_amount']
    E  = input_data['exposure']
    #I = input_data["pol_insee_code"]
    I = input_data[['pol_insee_code','latitude','longitude']]

    # features; note that the 'target' and sentive attribute columns are dropped
    X0 = input_data.iloc[:, 5:32]
    
    X = (X0
          .drop(columns=['pol_coverage','pol_insee_code'])
          .fillna('Unknown').pipe(pd.get_dummies))
    INSEE = input_data.iloc[:, 36]
    print(f"features X: {X.shape[0]} samples, {X.shape[1]} attributes")
    print(f"targets y: {y.shape[0]} samples")
    print(f"sensitives E: {E.shape[0]} samples, {E.shape[0]} attributes")
    return X, y, G, E, I

X, y, G, E, I =load_ICU_Pricing("http://grarivincent.com/research/db4modelfull3.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


(98417, 151)
features X: 98417 samples, 1167 attributes
targets y: 98417 samples
sensitives E: 98417 samples, 98417 attributes


In [ ]:
(y!=0).sum()

11025

In [ ]:
X=X[y>0]
G=G[y>0]
E=E[y>0]
I=I[y>0]
y=y[y>0]

In [ ]:
X.shape

(11025, 1167)

In [ ]:
(y>0).mean()

1.0

In [ ]:
y.mean()

1107.7378757369586

In [ ]:
G= G.iloc[:, :-5]

In [ ]:
X_train, X_test, y_train, y_test, G_train, G_test, E_train, E_test, I_train, I_test  = train_test_split(X, y, G, E, I, test_size=0.2, random_state=7)

In [ ]:
scaler = StandardScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)
X_train = X_train.pipe(scale_df, scaler) 
X_test = X_test.pipe(scale_df, scaler)

In [ ]:
scalerg = StandardScaler().fit(G_train)
scale_df_g = lambda df, scaler: pd.DataFrame(scalerg.transform(df), columns=df.columns, index=df.index)
G_train = G_train.pipe(scale_df_g, scalerg) 
G_test = G_test.pipe(scale_df_g, scalerg)

In [ ]:
y_traint =np.expand_dims(y_train,axis=1)
y_testt =np.expand_dims(y_test,axis=1)

In [ ]:
from sklearn import linear_model

### **GLM POISSON SANS ZONIER **

In [ ]:
from sklearn import linear_model
reg = linear_model.GammaRegressor() 
reg.fit(X_train,y_train)

GammaRegressor()

In [ ]:
def EDR_GAMMA(yhat, y):
  #loss=torch.mean(torch.exp(xbeta)-y*xbeta)
  #loss=torch.mean(yhat-y*torch.log(yhat))
  eps=0.000000000001
  res=1-np.mean(-np.log(y/yhat)+(y-yhat)/yhat)/np.mean(-np.log(y/np.mean(y))+(y-np.mean(y))/np.mean(y))
  return res

def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

In [ ]:
print(" GINI Poi without zonier via Reg : ", gini_coefficient(reg.predict(X_train)))
print(" GINI Poi without zonier via Reg : ", gini_coefficient(reg.predict(X_test)))
print(" EDR POIS Poi without zonier via Reg : ", EDR_GAMMA(reg.predict(X_train),y_train))
print(" EDR POIS Poi without zonier via Reg : ", EDR_GAMMA(reg.predict(X_test),y_test))
MSE_Poi_tr =  ((reg.predict(X_train)-y_train)**2).mean()
MSE_Poi_test =  ((reg.predict(X_test)-y_test)**2).mean()
print(" MSE Poi without zonier via Reg : ", MSE_Poi_tr)
print(" MSE Poi without zonier via Reg : ", MSE_Poi_test)

 GINI Poi without zonier via Reg :  0.10016313392464055
 GINI Poi without zonier via Reg :  0.09900343839239843
 EDR POIS Poi without zonier via Reg :  0.07082906969778546
 EDR POIS Poi without zonier via Reg :  0.009880909830848772
 MSE Poi without zonier via Reg :  3233947.3979483824
 MSE Poi without zonier via Reg :  19711010.204665985


In [ ]:
reg.score(X_train,y_train)

0.07082906969777525

In [ ]:
X_train.shape

(8820, 1167)

In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
def load_ICU_Pricing(path):
    #column_names = ["CalYear","Gender","Type","Category","Occupation","Age","Group1","Bonus","Poldur","Value","Adind","SubGroup2","Group2","Density","Y"]
    #input_data = (pd.read_csv(path, #names=column_names,
    #                           sep=r',', engine='python', keep_default_na=False, na_values=['NA']))
    
    path = "http://grarivincent.com/research/db4ModelFull2.csv"
    input_data = (pd.read_csv(path, #names=column_names,
                                sep=r',', engine='python', na_values=['NA']))
    input_data=input_data.drop(columns=['Unnamed: 0'])
    
    base=baseINSEE
    base = base[~base.pol_insee_code.str[:2].isin(["2A","2B"])]
    base["pol_insee_code"] = base["pol_insee_code"].astype(int)
    base_nd =  base.drop_duplicates(subset=['pol_insee_code'])

    input_data = input_data.drop_duplicates()
    input_data = input_data[~input_data.pol_insee_code.str[:2].isin(["2A","2B"])]
    input_data["pol_insee_code"] = input_data["pol_insee_code"].astype(int)

    input_data = input_data.merge(base_nd, how='inner', on="pol_insee_code")
    #input_data = input_data[input_data['claim_amount']<=5000]
    #input_data=input_data[input_data['claim_amount']!=0]
   
    #
    sensitive_attribs = ['Age']
    print(input_data.shape)
    G0 = (input_data.iloc[:, 37:180]) #137])
    #Z0 = pd.concat(Z0,input_data['long'])
    G0=G0.select_dtypes([np.number])
    G0 = G0.fillna(G0.mean())
    G=G0
    #Z = (Z0
    #    .drop(columns=['"REG"'
    #                   ])
    #    .fillna('Unknown').pipe(pd.get_dummies))

    y = input_data['claim_nb']
    E  = input_data['exposure']
    #I = input_data["pol_insee_code"]
    I = input_data[['pol_insee_code','latitude','longitude']]

    # features; note that the 'target' and sentive attribute columns are dropped
    X0 = input_data.iloc[:, 5:32]
    
    X = (X0
          .drop(columns=['pol_coverage','pol_insee_code'])
          .fillna('Unknown').pipe(pd.get_dummies))
    INSEE = input_data.iloc[:, 36]
    print(f"features X: {X.shape[0]} samples, {X.shape[1]} attributes")
    print(f"targets y: {y.shape[0]} samples")
    print(f"sensitives E: {E.shape[0]} samples, {E.shape[0]} attributes")
    return X, y, G, E, I

X, y, G, E, I =load_ICU_Pricing("http://grarivincent.com/research/db4modelfull3.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


(98417, 151)
features X: 98417 samples, 1167 attributes
targets y: 98417 samples
sensitives E: 98417 samples, 98417 attributes


In [ ]:
path = "http://grarivincent.com/research/db4ModelFull2.csv"
input_data = (pd.read_csv(path, #names=column_names,
                            sep=r',', engine='python', na_values=['NA']))
input_data=input_data.drop(columns=['Unnamed: 0'])

base=baseINSEE
base = base[~base.pol_insee_code.str[:2].isin(["2A","2B"])]
base["pol_insee_code"] = base["pol_insee_code"].astype(int)
base_nd =  base.drop_duplicates(subset=['pol_insee_code'])

input_data = input_data.drop_duplicates()
input_data = input_data[~input_data.pol_insee_code.str[:2].isin(["2A","2B"])]
input_data["pol_insee_code"] = input_data["pol_insee_code"].astype(int)

input_data = input_data.merge(base_nd, how='inner', on="pol_insee_code")
X0_f = input_data.iloc[:, 5:32]

X_f = (X0_f
      .drop(columns=['pol_coverage','pol_insee_code'])
      .fillna('Unknown').pipe(pd.get_dummies))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
(y>0).mean()

0.11202332930286434

In [ ]:
G= G.iloc[:, :-5]

In [ ]:
X_train, X_test, y_train, y_test, G_train, G_test, E_train, E_test, I_train, I_test  = train_test_split(X, y, G, E, I, test_size=0.2, random_state=7)

In [ ]:
scaler = StandardScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)
X_train = X_train.pipe(scale_df, scaler) 
X_test = X_test.pipe(scale_df, scaler)
X_f = X_f.pipe(scale_df, scaler)

In [ ]:
scalerg = StandardScaler().fit(G_train)
scale_df_g = lambda df, scalerg: pd.DataFrame(scalerg.transform(df), columns=df.columns, index=df.index)
G_train = G_train.pipe(scale_df_g, scalerg) 
G_test = G_test.pipe(scale_df_g, scalerg)

In [ ]:
y_traint =np.expand_dims(y_train,axis=1)
y_testt =np.expand_dims(y_test,axis=1)

### **GLM POISSON SANS ZONIER **

In [ ]:
# ICI il n'y a que des expsures en 1 d'ou l'offset (log(expo)) non présent 
from sklearn import linear_model
clf = linear_model.PoissonRegressor() 
clf.fit(X_train,y_train)

PoissonRegressor()

In [ ]:
def EDR_POIS(yhat, y):
  #loss=torch.mean(torch.exp(xbeta)-y*xbeta)
  #loss=torch.mean(yhat-y*torch.log(yhat))
  eps=0.000000000001
  res=1-np.mean((y*np.log((y+eps)/yhat)-(y-yhat)))/np.mean((y*np.log((y+eps)/np.mean(y))))
  return res

def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

In [ ]:
print(" GINI Poi without zonier via Reg : ", gini_coefficient(clf.predict(X_train)))
print(" GINI Poi without zonier via Reg : ", gini_coefficient(clf.predict(X_test)))
print(" EDR POIS Poi without zonier via Reg : ", EDR_POIS(clf.predict(X_train),y_train))
print(" EDR POIS Poi without zonier via Reg : ", EDR_POIS(clf.predict(X_test),y_test))
MSE_Poi_tr =  ((clf.predict(X_train)-y_train)**2).mean()
MSE_Poi_test =  ((clf.predict(X_test)-y_test)**2).mean()
print(" MSE Poi without zonier via Reg : ", MSE_Poi_tr)
print(" MSE Poi without zonier via Reg : ", MSE_Poi_test)

 GINI Poi without zonier via Reg :  0.08640736299232982
 GINI Poi without zonier via Reg :  0.08651112504694738
 EDR POIS Poi without zonier via Reg :  0.026092476605271142
 EDR POIS Poi without zonier via Reg :  0.021348395990629987
 MSE Poi without zonier via Reg :  0.1339346177925705
 MSE Poi without zonier via Reg :  0.13263329271680574


In [ ]:
FREQ = clf.predict(X_train)
CM= reg.predict(X_train)

In [ ]:
FREQ

array([0.11904605, 0.13147519, 0.12791458, ..., 0.11953604, 0.13345521,
       0.16311184])

In [ ]:
CM

array([ 870.11497723, 1007.00289405, 1041.24901349, ..., 1389.4234503 ,
       1106.50754425, 1167.91980885])

In [ ]:
X_final=input_data

In [ ]:
X_final['FREQ'] = clf.predict(X_f)
X_final['CM'] = reg.predict(X_f)
X_final['PP'] = X_final['FREQ']*X_final['CM']

In [ ]:
X_final['PP'].mean()

132.56949638415253

In [ ]:
input_data['claim_amount'].mean()

124.09248483493673

In [ ]:
X_final.shape

(98417, 154)

In [ ]:
input_data.iloc[:, 5:32].shape

(98417, 27)

In [ ]:
X_final=input_data.iloc[:, 5:32]

In [ ]:
X_final['FREQ'] = clf.predict(X_f)
X_final['CM'] = reg.predict(X_f)
X_final['PP'] = X_final['FREQ']*X_final['CM']

In [ ]:
X_final.shape

(98417, 30)

In [ ]:
X_final.to_csv('final.csv', header=True, index=None, sep=';')

124.09248483493673